# Read SQLite data from odds_data

This notebook discovers .db files under `odds_data/`, connects with sqlite3, lists tables, and previews rows safely without external libraries.

In [17]:
# Discover SQLite .db files under odds_data
from pathlib import Path

DATA_DIR = Path('odds_data')
db_files = sorted(DATA_DIR.glob('*.db'))

print(f"Found {len(db_files)} database file(s) in {DATA_DIR.resolve()}")
for i, f in enumerate(db_files, 1):
    print(f"{i}. {f.name}")

# Pick the first DB by default; change index to choose another
DB_INDEX = 1
if db_files:
    db_path = db_files[DB_INDEX-1]
    print(f"\nUsing database: {db_path}")
else:
    db_path = None
    print("No .db files found. Place a SQLite file in odds_data/.")

Found 1 database file(s) in /home/duy/project/bet/crawl_lol/odds_data
1. live_odds.db

Using database: odds_data/live_odds.db


In [18]:
# Inspect tables in the selected database
import sqlite3

if db_path is not None:
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
    tables = [r[0] for r in cur.fetchall()]
    print(f"Tables ({len(tables)}):", tables)
else:
    tables = []

Tables (2): ['live_odds', 'sqlite_sequence']


In [19]:
# Preview first rows from a table
from textwrap import shorten

if db_path is not None and tables:
    # Prefer the known table if present
    preferred = 'live_odds' if 'live_odds' in tables else tables[0]
    print(f"Previewing table: {preferred}")
    
    cur.execute(f"PRAGMA table_info({preferred})")
    cols = [c[1] for c in cur.fetchall()]
    print("Columns:", cols)
    
    cur.execute(f"SELECT * FROM {preferred} ORDER BY ROWID LIMIT 10")
    rows = cur.fetchall()
    print(f"\n{len(rows)} row(s):")
    for r in rows:
        print(r)
else:
    print("No table to preview.")

Previewing table: live_odds
Columns: ['id', 'timestamp', 'match_id', 'match_name', 'game_number', 'team1', 'team2', 'team1_odds', 'team2_odds', 'status', 'league', 'series_score', 'raw_data']

10 row(s):
(1, '2025-08-03T00:15:50.395756', 35256636, 'Game 1 Win (Live Odds)', 1, 'Fnatic', 'Team Heretics', 1.612, 2.273, '2', 'Unknown League', '0-0', '{"match_id": 35256636, "match_name": "Game 1 Win (Live Odds)", "team1": "Fnatic", "team2": "Team Heretics", "status": 2, "game_number": 1, "odds": {"Fnatic": 1.612, "Team Heretics": 2.273}}')
(2, '2025-08-03T00:15:53.658053', 35256636, 'Game 1 Win (Live Odds)', 1, 'Fnatic', 'Team Heretics', 1.612, 2.273, '2', 'Unknown League', '0-0', '{"match_id": 35256636, "match_name": "Game 1 Win (Live Odds)", "team1": "Fnatic", "team2": "Team Heretics", "status": 2, "game_number": 1, "odds": {"Fnatic": 1.612, "Team Heretics": 2.273}}')
(3, '2025-08-03T19:46:01.389867', 35250310, 'Game 2 Win (Live Odds)', 2, 'Top Esports', 'Bilibili Gaming', 16.409, 1.001, 

In [20]:
# Optional: load table into a pandas DataFrame
import pandas as pd

if db_path is not None and 'live_odds' in tables:
    q = """
    SELECT *
    FROM live_odds
    ORDER BY timestamp
    """
    df = pd.read_sql_query(q, conn)
    print(df.head())
    print("\nShape:", df.shape)

# Close connection when done
# if db_path is not None:
#     conn.close()

   id                   timestamp  match_id              match_name  \
0   7  2025-08-03T00:13:24.295620  35256636  Game 1 Win (Live Odds)   
1   8  2025-08-03T00:13:29.542185  35256636  Game 1 Win (Live Odds)   
2   9  2025-08-03T00:13:34.786517  35256636  Game 1 Win (Live Odds)   
3   1  2025-08-03T00:15:50.395756  35256636  Game 1 Win (Live Odds)   
4   2  2025-08-03T00:15:53.658053  35256636  Game 1 Win (Live Odds)   

   game_number   team1          team2  team1_odds  team2_odds status  \
0          1.0  Fnatic  Team Heretics       1.528       2.464      1   
1          1.0  Fnatic  Team Heretics       1.567       2.368      1   
2          1.0  Fnatic  Team Heretics       1.567       2.368      1   
3          1.0  Fnatic  Team Heretics       1.612       2.273      2   
4          1.0  Fnatic  Team Heretics       1.612       2.273      2   

           league series_score  \
0  Unknown League          0-0   
1  Unknown League          0-0   
2  Unknown League          0-0   
3  U

In [21]:
df.tail(100)

,id,timestamp,match_id,match_name,game_number,team1,team2,team1_odds,team2_odds,status,league,series_score,raw_data
2344,2345,2025-08-09T14:02:55.729253,35384955,Game 2 Win (Live Odds),2.0,OKSavingsBank BRION,DRX,1.627,2.245,1,Unknown League,0-1,"{""match_id"": 35384955, ""match_name"": ""Game 2 W..."
2345,2346,2025-08-09T14:02:59.000621,35384955,Game 2 Win (Live Odds),2.0,OKSavingsBank BRION,DRX,1.627,2.245,1,Unknown League,0-1,"{""match_id"": 35384955, ""match_name"": ""Game 2 W..."
2346,2347,2025-08-09T14:03:02.340531,35384955,Game 2 Win (Live Odds),2.0,OKSavingsBank BRION,DRX,1.627,2.245,1,Unknown League,0-1,"{""match_id"": 35384955, ""match_name"": ""Game 2 W..."
2347,2348,2025-08-09T14:03:05.661083,35384955,Game 2 Win (Live Odds),2.0,OKSavingsBank BRION,DRX,1.627,2.245,1,Unknown League,0-1,"{""match_id"": 35384955, ""match_name"": ""Game 2 W..."
2348,2349,2025-08-09T14:03:09.002392,35384955,Game 2 Win (Live Odds),2.0,OKSavingsBank BRION,DRX,1.602,2.293,1,Unknown League,0-1,"{""match_id"": 35384955, ""match_name"": ""Game 2 W..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2439,2440,2025-08-09T14:08:05.616758,35318379,Game 1 Win (Live Odds),1.0,Bilibili Gaming,Team WE,1.275,3.620,1,LPL 2025 Split 3,0-0,"{""match_id"": 35318379, ""match_name"": ""Game 1 W..."
2440,2441,2025-08-09T14:08:08.952405,35384955,Game 2 Win (Live Odds),2.0,OKSavingsBank BRION,DRX,1.705,2.111,1,LCK 2025 Season,0-1,"{""match_id"": 35384955, ""match_name"": ""Game 2 W..."
2441,2442,2025-08-09T14:08:09.014688,35318379,Game 1 Win (Live Odds),1.0,Bilibili Gaming,Team WE,1.275,3.620,1,LPL 2025 Split 3,0-0,"{""match_id"": 35318379, ""match_name"": ""Game 1 W..."
2442,2443,2025-08-09T14:08:12.269118,35384955,Game 2 Win (Live Odds),2.0,OKSavingsBank BRION,DRX,1.705,2.111,1,LCK 2025 Season,0-1,"{""match_id"": 35384955, ""match_name"": ""Game 2 W..."
